In [1]:
%load_ext autoreload
%autoreload 2
from lens_catalog import OM10LensCatalog
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from Utils.inference_utils import median_sigma_from_samples
import h5py
import pandas as pd
import corner
from matplotlib.lines import Line2D
import Utils.mcmc_utils as mcmc_utils

In [ ]:
gt_lenses = OM10LensCatalog('MassModels/om10_sample/om10_venkatraman_erickson24.csv')

### Define Gold/Silver Samples ###

We split the sample into doubles and quads. We already have a sample of ~30 quads (the STRIDES sample), so we take a conservative assumption of 50 quads in the gold sample. We add 200 doubles to the gold sample.
The remaining lenses are added to the silver sample. 

In [3]:
# split into doubles & quads
dbls = gt_lenses.doubles_indices()
quads = gt_lenses.quads_indices()
# Let's assume in the gold sample: 200 doubles, 50 quads (overall quad fraction is 11%, so this is amplified)
# The rest are silver (regardless of time delay)
dbls = gt_lenses.doubles_indices()
quads = gt_lenses.quads_indices()

gold_dbls = dbls[:200]
silver_dbls = dbls[200:]

gold_quads = quads[:50]
silver_quads = quads[50:]

gold = np.append(gold_dbls,gold_quads)
silver = np.append(silver_dbls,silver_quads)

### "Emulate" Time Delay Measurements ###

We assume a time delay measurement uncertainty of 2 days for the gold lenses (dedicated monitoring campaigns), and 5 days for the silver lenses (LSST light curves)

In [4]:
from scipy.stats import norm
gold_sigma = 2.
silver_sigma = 5.
# gold doubles 'measured' time-delays
mus = gt_lenses.lens_df.loc[gold_dbls,'td01']
sigma = np.ones(len(mus))*gold_sigma
gold_dbls_measured_td = np.zeros((len(mus),3))
gold_dbls_measured_td[:,0] = norm.rvs(loc=mus,scale=sigma)
gold_dbls_measured_prec = np.zeros((len(mus),3,3))
gold_dbls_measured_prec[:,0,0] += 1/(gold_sigma**2)
gold_dbls_prefactor = np.ones(len(mus))* (1/(2*np.pi))**(1/2) / gold_sigma

# silver doubles 'measured' time-delays
mus = gt_lenses.lens_df.loc[silver_dbls,'td01']
sigma = np.ones(len(mus))*silver_sigma
silver_dbls_measured_td = np.zeros((len(mus),3))
silver_dbls_measured_td[:,0] = norm.rvs(loc=mus,scale=sigma)
silver_dbls_measured_prec = np.zeros((len(mus),3,3))
silver_dbls_measured_prec[:,0,0] += 1/(silver_sigma**2)
silver_dbls_prefactor = np.ones(len(mus))* (1/(2*np.pi))**(1/2) / gold_sigma

# gold quads 'measured' time-delays
mus = gt_lenses.lens_df.loc[gold_quads,['td01','td02','td03']].to_numpy()
sigma = np.ones(len(mus))*gold_sigma
gold_quads_measured_td = np.ones((len(mus),3))*np.nan
for i in range(0,3):
    gold_quads_measured_td[:,i] = norm.rvs(loc=mus[:,i],scale=sigma)
gold_quads_measured_prec = np.eye(3,3)/(gold_sigma**2)
gold_quads_measured_prec = np.repeat(gold_quads_measured_prec[np.newaxis,:],len(mus), axis=0)
gold_quads_prefactor = np.ones(len(mus))* (1/(2*np.pi))**(3/2) / np.sqrt(np.linalg.det(np.eye(3,3)*(gold_sigma**2)))

# silver quads 'measured' time-delays
mus = gt_lenses.lens_df.loc[silver_quads,['td01','td02','td03']].to_numpy()
sigma = np.ones(len(mus))*silver_sigma
silver_quads_measured_td = np.ones((len(mus),3))*np.nan
for i in range(0,3):
    silver_quads_measured_td[:,i] = norm.rvs(loc=mus[:,i],scale=sigma)
silver_quads_measured_prec = np.eye(3,3)/(silver_sigma**2)
silver_quads_measured_prec = np.repeat(silver_quads_measured_prec[np.newaxis,:],len(mus), axis=0)
silver_quads_prefactor = np.ones(len(mus))* (1/(2*np.pi))**(3/2) / np.sqrt(np.linalg.det(np.eye(3,3)*(silver_sigma**2)))

### FPD & Gamma Samples ###

Using NPE mass models, return samples of fermat potential difference and $\gamma_{lens}$. Two different networks are trained for HST quality images (gold) and LSST quality images (silver). We'll track 5,000 samples of each.

Load in pre-computed NPE preds

In [5]:
hst_mu = np.load('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/MassModels/om10_sample/y_pred_hst.npy')
hst_cov = np.load('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/MassModels/om10_sample/cov_pred_hst.npy')

lsst_mu = np.load('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/MassModels/om10_sample/y_pred_lsst.npy')
lsst_cov = np.load('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/MassModels/om10_sample/cov_pred_lsst.npy')

Compute fpd samps,  track gamma samps

In [6]:
from scipy.stats import multivariate_normal
import batched_fermatpot
num_samps = 5000
# gold doubles
gold_dbls_fpd_samps = np.zeros((len(gold_dbls),num_samps,3))
gold_dbls_gamma_samps = np.empty((len(gold_dbls),num_samps))
for i in range(0,len(gold_dbls)):
    lens_param_samps = multivariate_normal.rvs(mean=hst_mu[gold_dbls[i]],
        cov=hst_cov[gold_dbls[i]],size=num_samps)
    x_im = gt_lenses.lens_df.loc[gold_dbls[i],['x_im0','x_im1']].to_numpy().astype(float)
    y_im = gt_lenses.lens_df.loc[gold_dbls[i],['y_im0','y_im1']].to_numpy().astype(float)
    fpd_samps = batched_fermatpot.eplshear_fp_samples(x_im,y_im,
        lens_param_samps[:,:8],lens_param_samps[:,8],lens_param_samps[:,9])
    gold_dbls_fpd_samps[i,:,0] = fpd_samps[:,0] - fpd_samps[:,1]
    gold_dbls_gamma_samps[i,:] = lens_param_samps[:,3]

# silver doubles
silver_dbls_fpd_samps = np.zeros((len(silver_dbls),num_samps,3))
silver_dbls_gamma_samps = np.empty((len(silver_dbls),num_samps))
for i in range(0,len(silver_dbls)):
    lens_param_samps = multivariate_normal.rvs(mean=hst_mu[silver_dbls[i]],
        cov=hst_cov[silver_dbls[i]],size=num_samps)
    x_im = gt_lenses.lens_df.loc[silver_dbls[i],['x_im0','x_im1']].to_numpy().astype(float)
    y_im = gt_lenses.lens_df.loc[silver_dbls[i],['y_im0','y_im1']].to_numpy().astype(float)
    fpd_samps = batched_fermatpot.eplshear_fp_samples(x_im,y_im,
        lens_param_samps[:,:8],lens_param_samps[:,8],lens_param_samps[:,9])
    silver_dbls_fpd_samps[i,:,0] = fpd_samps[:,0] - fpd_samps[:,1]
    silver_dbls_gamma_samps[i,:] = lens_param_samps[:,3]


# gold quads
gold_quads_fpd_samps = np.zeros((len(gold_quads),num_samps,3))
gold_quads_gamma_samps = np.empty((len(gold_quads),num_samps))
gold_quads_fpd_samps_EM = np.zeros((len(gold_quads),num_samps,3))
gold_quads_gamma_samps_EM = np.empty((len(gold_quads),num_samps))

for i in range(0,len(gold_quads)):
    lens_param_samps = multivariate_normal.rvs(mean=hst_mu[gold_quads[i]],
        cov=hst_cov[gold_quads[i]],size=num_samps)
    
    # TODO: replace lens_params_samps for a gold_quads debugging test
    mu_em = gt_lenses.lens_df.loc[gold_quads[i],['theta_E', 'gamma1', 'gamma2', 'gamma', 'e1', 'e2', 
            'center_x', 'center_y', 'src_center_x','src_center_y']].to_numpy().astype(float)
    sigmas_em = np.asarray([0.01,0.01,0.01,0.06,0.02,0.02,0.005,0.005,0.005,0.005])
    cov_em = np.diag(sigmas_em**2)
    lens_params_samps_em = multivariate_normal.rvs(mean=mu_em,cov=cov_em,size=num_samps)

    x_im = gt_lenses.lens_df.loc[gold_quads[i],['x_im0','x_im1','x_im2','x_im3']].to_numpy().astype(float)
    y_im = gt_lenses.lens_df.loc[gold_quads[i],['y_im0','y_im1','y_im2','y_im3']].to_numpy().astype(float)

    fpd_samps = batched_fermatpot.eplshear_fp_samples(x_im,y_im,
        lens_param_samps[:,:8],lens_param_samps[:,8],lens_param_samps[:,9])
    gold_quads_fpd_samps[i,:,0] = fpd_samps[:,0] - fpd_samps[:,1]
    gold_quads_fpd_samps[i,:,1] = fpd_samps[:,0] - fpd_samps[:,2]
    gold_quads_fpd_samps[i,:,2] = fpd_samps[:,0] - fpd_samps[:,3]
    gold_quads_gamma_samps[i,:] = lens_param_samps[:,3]

    fpd_samps_em = batched_fermatpot.eplshear_fp_samples(x_im,y_im,
        lens_params_samps_em[:,:8],lens_params_samps_em[:,8],lens_params_samps_em[:,9])
    gold_quads_fpd_samps_EM[i,:,0] = fpd_samps_em[:,0] - fpd_samps_em[:,1]
    gold_quads_fpd_samps_EM[i,:,1] = fpd_samps_em[:,0] - fpd_samps_em[:,2]
    gold_quads_fpd_samps_EM[i,:,2] = fpd_samps_em[:,0] - fpd_samps_em[:,3]
    gold_quads_gamma_samps_EM[i,:] = lens_params_samps_em[:,3]
    

# silver quads
silver_quads_fpd_samps = np.zeros((len(silver_quads),num_samps,3))
silver_quads_gamma_samps = np.empty((len(silver_quads),num_samps))
for i in range(0,len(silver_quads)):
    lens_param_samps = multivariate_normal.rvs(mean=hst_mu[silver_quads[i]],
        cov=hst_cov[silver_quads[i]],size=num_samps)
    x_im = gt_lenses.lens_df.loc[silver_quads[i],['x_im0','x_im1','x_im2','x_im3']].to_numpy().astype(float)
    y_im = gt_lenses.lens_df.loc[silver_quads[i],['y_im0','y_im1','y_im2','y_im3']].to_numpy().astype(float)
    fpd_samps = batched_fermatpot.eplshear_fp_samples(x_im,y_im,
        lens_param_samps[:,:8],lens_param_samps[:,8],lens_param_samps[:,9])
    silver_quads_fpd_samps[i,:,0] = fpd_samps[:,0] - fpd_samps[:,1]
    silver_quads_fpd_samps[i,:,1] = fpd_samps[:,0] - fpd_samps[:,2]
    silver_quads_fpd_samps[i,:,2] = fpd_samps[:,0] - fpd_samps[:,3]
    silver_quads_gamma_samps[i,:] = lens_param_samps[:,3]

### Save all data vectors to an *informatively named* folder ###

In [8]:
exp_folder_name = 'gold2days_silver5days'

# let's dump everything into .h5 files

h5f = h5py.File(('DataVectors/'+exp_folder_name+'/gold_quads.h5'), 'w')
h5f.create_dataset('measured_td', data=gold_quads_measured_td)
h5f.create_dataset('measured_prec',data=gold_quads_measured_prec)
h5f.create_dataset('prefactor',data=gold_quads_prefactor)
h5f.create_dataset('fpd_samps',data=gold_quads_fpd_samps)
h5f.create_dataset('gamma_samps',data=gold_quads_gamma_samps)
h5f.create_dataset('z_lens',data=gt_lenses.lens_df.loc[gold_quads,'z_lens'].to_numpy().astype(float))
h5f.create_dataset('z_src',data=gt_lenses.lens_df.loc[gold_quads,'z_src'].to_numpy().astype(float))
h5f.close()

h5f = h5py.File(('DataVectors/'+exp_folder_name+'/gold_dbls.h5'), 'w')
h5f.create_dataset('measured_td', data=gold_dbls_measured_td)
h5f.create_dataset('measured_prec',data=gold_dbls_measured_prec)
h5f.create_dataset('prefactor',data=gold_dbls_prefactor)
h5f.create_dataset('fpd_samps',data=gold_dbls_fpd_samps)
h5f.create_dataset('gamma_samps',data=gold_dbls_gamma_samps)
h5f.create_dataset('z_lens',data=gt_lenses.lens_df.loc[gold_dbls,'z_lens'].to_numpy().astype(float))
h5f.create_dataset('z_src',data=gt_lenses.lens_df.loc[gold_dbls,'z_src'].to_numpy().astype(float))
h5f.close()

h5f = h5py.File(('DataVectors/'+exp_folder_name+'/silver_quads.h5'), 'w')
h5f.create_dataset('measured_td', data=silver_quads_measured_td)
h5f.create_dataset('measured_prec',data=silver_quads_measured_prec)
h5f.create_dataset('prefactor',data=silver_quads_prefactor)
h5f.create_dataset('fpd_samps',data=silver_quads_fpd_samps)
h5f.create_dataset('gamma_samps',data=silver_quads_gamma_samps)
h5f.create_dataset('z_lens',data=gt_lenses.lens_df.loc[silver_quads,'z_lens'].to_numpy().astype(float))
h5f.create_dataset('z_src',data=gt_lenses.lens_df.loc[silver_quads,'z_src'].to_numpy().astype(float))
h5f.close()

h5f = h5py.File(('DataVectors/'+exp_folder_name+'/silver_dbls.h5'), 'w')
h5f.create_dataset('measured_td', data=silver_dbls_measured_td)
h5f.create_dataset('measured_prec',data=silver_dbls_measured_prec)
h5f.create_dataset('prefactor',data=silver_dbls_prefactor)
h5f.create_dataset('fpd_samps',data=silver_dbls_fpd_samps)
h5f.create_dataset('gamma_samps',data=silver_dbls_gamma_samps)
h5f.create_dataset('z_lens',data=gt_lenses.lens_df.loc[silver_dbls,'z_lens'].to_numpy().astype(float))
h5f.create_dataset('z_src',data=gt_lenses.lens_df.loc[silver_dbls,'z_src'].to_numpy().astype(float))
h5f.close()

### Let's Try to use TDC Sampler ###

We'll just use the gold quads, since this is a small number of lenses, so the inference can run in a reasonable amount of time. These lenses are also the most constraining, which is useful for debugging.

In [ ]:
import tdc_sampler

# let's just try with gold quads
my_tdc = tdc_sampler.TDCLikelihood(gold_quads_measured_td,gold_quads_measured_prec,
    gold_quads_prefactor,gold_quads_fpd_samps,gold_quads_gamma_samps,
    gt_lenses.lens_df.loc[gold_quads,'z_lens'].to_numpy().astype(float),
    gt_lenses.lens_df.loc[gold_quads,'z_src'].to_numpy().astype(float),
    log_prob_modeling_prior='hardcoded')

mcmc_chain_quads = tdc_sampler.fast_TDC(my_tdc,num_emcee_samps=1000,
    cosmo_model='LCDM')

In [ ]:
np.max(np.abs(gold_quads_measured_td[2]))

In [ ]:
# w0,wa Gold Quads
# let's just try with gold quads
my_tdc = tdc_sampler.TDCLikelihood(gold_quads_measured_td,gold_quads_measured_prec,
    gold_quads_prefactor,gold_quads_fpd_samps,gold_quads_gamma_samps,
    gt_lenses.lens_df.loc[gold_quads,'z_lens'].to_numpy().astype(float),
    gt_lenses.lens_df.loc[gold_quads,'z_src'].to_numpy().astype(float),
    log_prob_modeling_prior='hardcoded',cosmo_model='w0waCDM')

mcmc_chain_quads_w0wa_1day = tdc_sampler.fast_TDC(my_tdc,cosmo_model='w0waCDM',
    num_emcee_samps=6000,n_walkers=20)

In [ ]:
# w0,wa Gold Quads
# let's just try with gold quads
my_tdc = tdc_sampler.TDCLikelihood(gold_quads_measured_td,gold_quads_measured_prec,
    gold_quads_prefactor,gold_quads_fpd_samps_EM,gold_quads_gamma_samps_EM,
    gt_lenses.lens_df.loc[gold_quads,'z_lens'].to_numpy().astype(float),
    gt_lenses.lens_df.loc[gold_quads,'z_src'].to_numpy().astype(float),
    log_prob_modeling_prior='hardcoded',cosmo_model='w0waCDM')

mcmc_chain_quads_w0wa_1day_EM = tdc_sampler.fast_TDC(my_tdc,cosmo_model='w0waCDM',
    num_emcee_samps=6000,n_walkers=20)

In [ ]:
# w0,wa Gold Quads + Gold doubles

gold_measured_td = np.vstack((
    gold_quads_measured_td,
    gold_dbls_measured_td))
gold_measured_prec = np.vstack((
    gold_quads_measured_prec,
    gold_dbls_measured_prec
))
gold_prefactor = np.append(
    gold_quads_prefactor,
    gold_dbls_prefactor
)
gold_fpd_samps = np.vstack((
    gold_quads_fpd_samps,
    gold_dbls_fpd_samps
))
gold_gamma_samps = np.vstack((
    gold_quads_gamma_samps,
    gold_dbls_gamma_samps
))

gold_idxs = np.hstack((gold_quads,gold_dbls))



# let's just try with gold quads
my_tdc = tdc_sampler.TDCLikelihood(gold_measured_td,gold_measured_prec,
    gold_prefactor,gold_fpd_samps,gold_gamma_samps,
    gt_lenses.lens_df.loc[gold_idxs,'z_lens'].to_numpy().astype(float),
    gt_lenses.lens_df.loc[gold_idxs,'z_src'].to_numpy().astype(float),
    log_prob_modeling_prior='hardcoded',cosmo_model='w0waCDM')

mcmc_chain_quads_w0wa_200 = tdc_sampler.fast_TDC(my_tdc,cosmo_model='w0waCDM',
    num_emcee_samps=6000,n_walkers=20)

### Check Chains for Gold Quads, LCDM vs. w0waCDM ###

In [ ]:
mu_mean_quad = np.mean(gt_lenses.lens_df.loc[gold_quads,'gamma'].to_numpy().astype(float))
std_mean_quad = np.std(gt_lenses.lens_df.loc[gold_quads,'gamma'].to_numpy().astype(float),ddof=1)


exp_chains = [mcmc_chain_quads]
exp_names = ['ML Posteriors, Gold Quads']
burnin = 400
colors = ['gold']
custom_lines = []
custom_labels = []
for i,exp_chain in enumerate(exp_chains):
     
    if i ==0:

        figure = corner.corner(exp_chain[:,burnin:,:].reshape((-1,3)),plot_datapoints=False,
                    color=colors[i],levels=[0.68,0.95],fill_contours=True,
                    labels=['$H_0$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
                    dpi=200,truths=[70.,mu_mean_quad,std_mean_quad],truth_color='black',
                    fig=None,label_kwargs={'fontsize':17})

    else:

        corner.corner(exp_chain[:,burnin:,:].reshape((-1,3)),plot_datapoints=False,
                    color=colors[i],levels=[0.68,0.95],fill_contours=True,
                    labels=['$H_0$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
                    truths=[70.,mu_mean_quad,std_mean_quad],truth_color='black',
                    fig=figure,label_kwargs={'fontsize':17})
        
    custom_lines.append(Line2D([0], [0], color=colors[i], lw=4))

    # calculate h0 constraint
    h0, h0_sigma = median_sigma_from_samples(exp_chain[:,burnin:,0].reshape((-1,1)),weights=None)
    # construct label
    custom_labels.append(exp_names[i]+':\n $H_0$=%.2f$\pm$%.2f'%(h0, h0_sigma))


"""
axes = np.array(figure.axes).reshape((3, 3))
bounds = [[63,77],[1.91,2.095],[0.0,0.2]]
for r in range(0,3):
        for c in range(0,r+1):
            if bounds is not None:
                axes[r,c].set_xlim(bounds[c])
                if r != c :
                    axes[r,c].set_ylim(bounds[r])

axes = np.array(figure.axes).reshape((3, 3))
"""

axes = np.array(figure.axes).reshape((3, 3))
axes[0,2].legend(custom_lines,custom_labels,frameon=False,fontsize=13)

In [ ]:
np.append(silver,gold).shape

In [ ]:
h5f = h5py.File('DataVectors/gold2days_silver5days/gold_chain.h5', 'r')
gold_chain = h5f.get('mcmc_chain')[:]
h5f.close()

h5f = h5py.File('DataVectors/gold2days_silver5days/silver_chain.h5', 'r')
silver_chain = h5f.get('mcmc_chain')[:]
h5f.close()

h5f = h5py.File('DataVectors/gold2days_silver5days/gold_chain1000.h5', 'r')
gold_chain1000 = h5f.get('mcmc_chain')[:]
h5f.close()


mu_mean_gold = np.mean(gt_lenses.lens_df.loc[gold,'gamma'].to_numpy().astype(float))
std_mean_gold = np.std(gt_lenses.lens_df.loc[gold,'gamma'].to_numpy().astype(float),ddof=1)

mu_mean_silver = np.mean(gt_lenses.lens_df.loc[np.append(silver,gold),'gamma'].to_numpy().astype(float))
std_mean_silver = np.std(gt_lenses.lens_df.loc[np.append(silver,gold),'gamma'].to_numpy().astype(float),ddof=1)

exp_chains = [gold_chain,gold_chain1000]#mcmc_chain_quads_w0wa, mcmc_chain_quads_w0wa_1day,mcmc_chain_quads_w0wa_1day_EM, mcmc_chain_quads_w0wa_200]
exp_names = ['Gold: 50 Quads+ 200 Dbls, NPE Models, 5e3 fpd samps', #2-day td error
             'Gold: 50 Quads+ 200 Dbls, NPE Models, 1e3 fpd samps',
             '250Gold+1034Silver, NPE Models, 5-day silver TD error']#'50 Gold Quads, NPE Models, 2-day td error', 
             #'50 Gold Quads, NPE Models, 1-day td error', 
             #'50 Gold Quads, Emulated Models, 1-day td error',
             #'Gold: 50 Quads+ 200 Dbls, NPE Models, 1-day td error']
burnin = int(2e3)
colors = ['purple','turquoise']#'gold','silver','indianred','turquoise','purple']
custom_labels = []

custom_lines = []
custom_labels = []
for i,exp_chain in enumerate(exp_chains):
     
    if i ==0:

        figure = corner.corner(exp_chain[:,burnin:,:].reshape((-1,5)),plot_datapoints=False,
            color=colors[i],levels=[0.68,0.95],fill_contours=True,
            labels=['$H_0$','w$_0$','w$_a$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
            dpi=200,truths=[70.,-1.,0.,mu_mean_gold,std_mean_gold],truth_color='black',
            fig=None,label_kwargs={'fontsize':17})

    else:

        corner.corner(exp_chain[:,burnin:,:].reshape((-1,5)),plot_datapoints=False,
            color=colors[i],levels=[0.68,0.95],fill_contours=True,
            labels=['$H_0$','w$_0$','w$_a$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
            dpi=200,truths=[70.,-1.,0.,mu_mean_silver,std_mean_silver],truth_color='black',
            fig=figure,label_kwargs={'fontsize':17})
        
    custom_lines.append(Line2D([0], [0], color=colors[i], lw=4))

    # calculate h0 constraint
    h0, h0_sigma = median_sigma_from_samples(exp_chain[:,burnin:,0].reshape((-1,1)),weights=None)
    # construct label
    custom_labels.append(exp_names[i]+':\n $H_0$=%.2f$\pm$%.2f'%(h0, h0_sigma))

"""
axes = np.array(figure.axes).reshape((3, 3))
bounds = [[63,77],[1.91,2.095],[0.0,0.2]]
for r in range(0,3):
        for c in range(0,r+1):
            if bounds is not None:
                axes[r,c].set_xlim(bounds[c])
                if r != c :
                    axes[r,c].set_ylim(bounds[r])

axes = np.array(figure.axes).reshape((3, 3))
"""

axes = np.array(figure.axes).reshape((5, 5))
axes[0,4].legend(custom_lines,custom_labels,frameon=False,fontsize=16)

In [ ]:
plt.hist(gt_lenses.lens_df.loc[np.append(silver,gold),'gamma'].to_numpy().astype(float),bins=20,density=True)
my_x = np.arange(1.55,2.6,0.01)
from scipy.stats import norm
plt.plot(my_x,norm.pdf(my_x,loc=mu_mean_silver,scale=std_mean_silver))

In [ ]:
param_labels = np.asarray(['$H_0$','w$_0$','w$_a$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'])
true_hyperparameters = np.asarray([70.,-1.,0.,mu_mean_quad,std_mean_quad])

mcmc_utils.analyze_chains(gold_chain,param_labels,true_hyperparameters,
                    'test.txt',show_chains=True,
                    burnin=int(1e3))

### TDC Sampler Gold Doubles & Quads!! ###

In [ ]:
# let's just try with gold doubles
my_tdc = tdc_sampler.TDCLikelihood(gold_dbls_measured_td,gold_dbls_measured_prec,
    gold_dbls_prefactor,gold_dbls_fpd_samps,gold_dbls_gamma_samps,
    gt_lenses.lens_df.loc[gold_dbls,'z_lens'].to_numpy().astype(float),
    gt_lenses.lens_df.loc[gold_dbls,'z_src'].to_numpy().astype(float),
    log_prob_modeling_prior='hardcoded')

mcmc_chain_dbls = tdc_sampler.fast_TDC(my_tdc,num_emcee_samps=1000,
    cosmo_model='LCDM')

In [ ]:
# let's just try with gold quads

gold_idxs = np.append(gold_quads,gold_dbls)

my_tdc = tdc_sampler.TDCLikelihood(
    np.vstack((gold_quads_measured_td,gold_dbls_measured_td)),
    np.vstack((gold_quads_measured_prec,gold_dbls_measured_prec)),
    np.append(gold_quads_prefactor,gold_dbls_prefactor),
    np.vstack((gold_quads_fpd_samps,gold_dbls_fpd_samps)),
    np.vstack((gold_quads_gamma_samps,gold_dbls_gamma_samps)),
    gt_lenses.lens_df.loc[gold_idxs,'z_lens'].to_numpy().astype(float),
    gt_lenses.lens_df.loc[gold_idxs,'z_src'].to_numpy().astype(float),
    log_prob_modeling_prior='hardcoded')

mcmc_chain_comb = tdc_sampler.fast_TDC(my_tdc,num_emcee_samps=1000,
    cosmo_model='LCDM')

In [ ]:
mu_mean_quad = np.mean(gt_lenses.lens_df.loc[gold_quads,'gamma'].to_numpy().astype(float))
mu_mean_double = np.mean(gt_lenses.lens_df.loc[gold_dbls,'gamma'].to_numpy().astype(float))
mu_mean_gold = np.mean(gt_lenses.lens_df.loc[gold_idxs,'gamma'].to_numpy().astype(float))

exp_chains = [mcmc_chain_quads,mcmc_chain_EM]
exp_names = ['ML Posteriors, Gold Quads','Emulated Posteriors, Gold Quads']
burnin = 200
colors = ['gold','skyblue','silver','skyblue']
custom_lines = []
custom_labels = []
for i,exp_chain in enumerate(exp_chains):
     
    if i ==0:

        figure = corner.corner(exp_chain[:,burnin:,:].reshape((-1,3)),plot_datapoints=False,
                    color=colors[i],levels=[0.68,0.95],fill_contours=True,
                    labels=['$H_0$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
                    dpi=200,truths=[70.,mu_mean_quad,-1],truth_color='black',
                    fig=None,label_kwargs={'fontsize':17})

    else:

        corner.corner(exp_chain[:,burnin:,:].reshape((-1,3)),plot_datapoints=False,
                    color=colors[i],levels=[0.68,0.95],fill_contours=True,
                    labels=['$H_0$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
                    truths=[70.,mu_mean_quad,-1],truth_color='black',
                    fig=figure,label_kwargs={'fontsize':17})
        
    custom_lines.append(Line2D([0], [0], color=colors[i], lw=4))

    # calculate h0 constraint
    h0, h0_sigma = median_sigma_from_samples(exp_chain[:,burnin:,0].reshape((-1,1)),weights=None)
    # construct label
    custom_labels.append(exp_names[i]+':\n $H_0$=%.2f$\pm$%.2f'%(h0, h0_sigma))


"""
axes = np.array(figure.axes).reshape((3, 3))
bounds = [[63,77],[1.91,2.095],[0.0,0.2]]
for r in range(0,3):
        for c in range(0,r+1):
            if bounds is not None:
                axes[r,c].set_xlim(bounds[c])
                if r != c :
                    axes[r,c].set_ylim(bounds[r])

axes = np.array(figure.axes).reshape((3, 3))
"""

axes = np.array(figure.axes).reshape((3, 3))
axes[0,2].legend(custom_lines,custom_labels,frameon=False,fontsize=13)

In [ ]:
mu_mean_quad = np.mean(gt_lenses.lens_df.loc[gold_quads,'gamma'].to_numpy().astype(float))
mu_mean_double = np.mean(gt_lenses.lens_df.loc[gold_dbls,'gamma'].to_numpy().astype(float))
mu_mean_gold = np.mean(gt_lenses.lens_df.loc[gold_idxs,'gamma'].to_numpy().astype(float))

exp_chains = [mcmc_chain_quads,mcmc_chain_dbls,mcmc_chain_comb]
exp_names = ['Gold Quads','Gold Doubles','Gold Quads+Doubles']
burnin = 400
colors = ['gold','skyblue','salmon','skyblue']
custom_lines = []
custom_labels = []
for i,exp_chain in enumerate(exp_chains):
     
    if i ==0:

        figure = corner.corner(exp_chain[:,burnin:,:].reshape((-1,3)),plot_datapoints=False,
                    color=colors[i],levels=[0.68,0.95],fill_contours=True,
                    labels=['$H_0$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
                    dpi=200,truths=[70.,-1,-1],truth_color='black',
                    fig=None,label_kwargs={'fontsize':17})

    else:

        corner.corner(exp_chain[:,burnin:,:].reshape((-1,3)),plot_datapoints=False,
                    color=colors[i],levels=[0.68,0.95],fill_contours=True,
                    labels=['$H_0$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
                    truths=[70.,-1,-1],truth_color='black',
                    fig=figure,label_kwargs={'fontsize':17})
        
    custom_lines.append(Line2D([0], [0], color=colors[i], lw=4))

    # calculate h0 constraint
    h0, h0_sigma = median_sigma_from_samples(exp_chain[:,burnin:,0].reshape((-1,1)),weights=None)
    # construct label
    custom_labels.append(exp_names[i]+':\n $H_0$=%.2f$\pm$%.2f'%(h0, h0_sigma))


"""
axes = np.array(figure.axes).reshape((3, 3))
bounds = [[63,77],[1.91,2.095],[0.0,0.2]]
for r in range(0,3):
        for c in range(0,r+1):
            if bounds is not None:
                axes[r,c].set_xlim(bounds[c])
                if r != c :
                    axes[r,c].set_ylim(bounds[r])

axes = np.array(figure.axes).reshape((3, 3))
"""

axes = np.array(figure.axes).reshape((3, 3))
axes[0,2].legend(custom_lines,custom_labels,frameon=False,fontsize=13)

## Let's Look at w0waCDM ##

In [ ]:
mu_mean_gold = np.mean(gt_lenses.lens_df.loc[gold_idxs,'gamma'].to_numpy().astype(float))

h5f = h5py.File('new_mass_models/gold_chain.h5', 'r')
gold_chain = h5f.get('mcmc_chain')[:]
h5f.close()

h5f = h5py.File('new_mass_models/silver_chain.h5', 'r')
silver_chain = h5f.get('mcmc_chain')[:]
h5f.close()

exp_chains = [gold_chain, silver_chain]
exp_names = ['Gold, 70 Lenses', 'Gold+Silver, 1284 Lenses ']
burnin = 400
colors = ['gold','skyblue','salmon','skyblue']
custom_lines = []
custom_labels = []

custom_lines = []
custom_labels = []
for i,exp_chain in enumerate(exp_chains):
     
    if i ==0:

        figure = corner.corner(exp_chain[:,burnin:,:].reshape((-1,5)),plot_datapoints=False,
            color=colors[i],levels=[0.68,0.95],fill_contours=True,
            labels=['$H_0$','w$_0$','w$_a$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
            dpi=200,truths=[70.,-1.,0.,-1,-1],truth_color='black',
            fig=None,label_kwargs={'fontsize':17})

    else:

        corner.corner(exp_chain[:,burnin:,:].reshape((-1,5)),plot_datapoints=False,
            color=colors[i],levels=[0.68,0.95],fill_contours=True,
            labels=['$H_0$','w$_0$','w$_a$',r'$\mu(\gamma_{lens})$',r'$\sigma(\gamma_{lens})$'],
            dpi=200,truths=[70.,-1.,0.,-1,-1],truth_color='black',
            fig=figure,label_kwargs={'fontsize':17})
        
    custom_lines.append(Line2D([0], [0], color=colors[i], lw=4))

    # calculate h0 constraint
    h0, h0_sigma = median_sigma_from_samples(exp_chain[:,burnin:,0].reshape((-1,1)),weights=None)
    # construct label
    custom_labels.append(exp_names[i]+':\n $H_0$=%.2f$\pm$%.2f'%(h0, h0_sigma))

"""
axes = np.array(figure.axes).reshape((3, 3))
bounds = [[63,77],[1.91,2.095],[0.0,0.2]]
for r in range(0,3):
        for c in range(0,r+1):
            if bounds is not None:
                axes[r,c].set_xlim(bounds[c])
                if r != c :
                    axes[r,c].set_ylim(bounds[r])

axes = np.array(figure.axes).reshape((3, 3))
"""

axes = np.array(figure.axes).reshape((5, 5))
axes[0,4].legend(custom_lines,custom_labels,frameon=False,fontsize=16)

## Sanity Checks ##

### Does the assumed training prior match the effective training prior after cuts ###

In [ ]:
hst_val = pd.read_csv('MassModels/hst_validation_metadata.csv')
gamma_vals = hst_val['main_deflector_parameters_gamma'].to_numpy().astype(float)
plt.hist(gamma_vals,density=True,label='Samples')
x_range = np.arange(1.4,2.6,0.01)
mu_est = np.mean(gamma_vals)
sigma_est = np.std(gamma_vals,ddof=1)
plt.plot(x_range,norm.pdf(x_range,mu_est,sigma_est),label='$\mu=%.3f$, $\sigma=%.2f$'%(mu_est,sigma_est))
plt.legend()
plt.title('Validation')

plt.figure()
hst_train0 = pd.read_csv('MassModels/hst_train0_metadata.csv')
gamma_vals = hst_train0['main_deflector_parameters_gamma'].to_numpy().astype(float)
plt.hist(gamma_vals,density=True,label='Samples')
x_range = np.arange(1.4,2.6,0.01)
mu_est = np.mean(gamma_vals)
sigma_est = np.std(gamma_vals,ddof=1)
plt.plot(x_range,norm.pdf(x_range,mu_est,sigma_est),label='$\mu=%.3f$, $\sigma=%.2f$'%(mu_est,sigma_est))
plt.legend()
plt.title('Train 0')

from scipy.stats import chisquare
from scipy.stats import gaussian_kde
plt.figure()

gaussian_samples = norm.rvs(mu_est,sigma_est,size=10000)

# Let's try a KDE
gamma_kde = gaussian_kde(gamma_vals)
kde_samples = gamma_kde.resample(size=10000)[0]
bins=np.histogram(np.hstack((kde_samples,gamma_vals)), bins=40)[1]

counts_exp,_,_ = plt.hist(kde_samples,bins,
            histtype='step',label='KDE Estimate')
counts_obs,_,_ = plt.hist(gamma_vals,bins,histtype='step',
                          label='Training Samples')
    
# only take bins where counts are greater than 5, then add in an extra bin at beginning and end for the tails
idx = np.where((counts_exp > 5) & (counts_obs > 5))[0]
# less than some #, bins, greather than some #
prepend = np.sum(counts_obs[:idx[0]])
append = np.sum(counts_obs[(idx[-1]+1):])
counts_obs_final = np.concatenate(([prepend],counts_obs[idx],[append]))
prepend = np.sum(counts_exp[:idx[0]])
append = np.sum(counts_exp[(idx[-1]+1):])
counts_exp_final = np.concatenate(([prepend],counts_exp[idx],[append]))



chi2_distance = np.sum((counts_obs_final-counts_exp_final)**2/counts_exp_final)
#print(chi2_distance)

chi2,_= chisquare(counts_obs_final,counts_exp_final)
print(chi2)

dof = len(counts_obs_final) - 1
print('chi2/dof:', chi2/dof)
plt.text(1.6,800,r'$\frac{\chi^2}{\nu}$: %.2f'%(chi2/dof),
                        {'fontsize':13})
plt.legend()

In [ ]:
num_emcee_samps = 6000
exp_folder = 'gold2days_silver5days'

lens_types = ['gold_quads','gold_dbls','silver_quads','silver_dbls']

inputs_dict = {
    'gold_quads':{},
    'gold_dbls':{},
    'silver_quads':{},
    'silver_dbls':{}
}

input_keys = ['measured_td','measured_prec','prefactor','fpd_samps','gamma_samps','z_lens','z_src']

for l in lens_types:

    my_filepath = ('/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/'+
        'DataVectors/'+exp_folder+'/'+l+'.h5')
    h5f = h5py.File(my_filepath, 'r')
    for key in input_keys:
        inputs_dict[l][key] = h5f.get(key)[:]
    h5f.close()


# let's run gold-only and gold+silver
gold_measured_td = np.vstack((
    inputs_dict['gold_quads']['measured_td'],
    inputs_dict['gold_dbls']['measured_td']))
gold_measured_prec = np.vstack((
    inputs_dict['gold_quads']['measured_prec'],
    inputs_dict['gold_dbls']['measured_prec']
))
gold_prefactor = np.append(
    inputs_dict['gold_quads']['prefactor'],
    inputs_dict['gold_dbls']['prefactor']
)
# TODO: try only 1,000 samps here
gold_fpd_samps = np.vstack((
    inputs_dict['gold_quads']['fpd_samps'],
    inputs_dict['gold_dbls']['fpd_samps']
))




In [ ]:
inputs_dict['gold_quads']['prefactor'].shape

In [ ]:
inputs_dict['gold_dbls']['fpd_samps'][:,:1000,:].shape